In [1]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



device = 'cpu'

#Code loosly based on https://datascience.stackexchange.com/questions/56804/sckit-learn-cross-validation-and-model-retrain
#And our exercise 1a

In [2]:
train = pd.read_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\task1b_ql4jfi6af0\train.csv')

X = train.iloc[:,2:]
Y = train.iloc[:,1]
ID = train.iloc[:,0]
pd.set_option("display.precision", 7)
print(X)
print(Y)
print(X.shape)
print(Y.shape)

       x1    x2    x3    x4    x5
0    0.02  0.05 -0.09 -0.43 -0.08
1   -0.13  0.11 -0.08 -0.29 -0.03
2    0.08  0.06 -0.07 -0.41 -0.03
3    0.02 -0.12  0.01 -0.43 -0.02
4   -0.14 -0.12 -0.08 -0.02 -0.08
..    ...   ...   ...   ...   ...
695  0.31 -0.09  0.04 -0.09  0.03
696 -0.26 -0.01  0.02 -0.40  0.05
697 -0.27 -0.22 -0.01 -0.32 -0.05
698  0.19  0.11 -0.05 -0.27 -0.04
699 -0.09 -0.09 -0.06 -0.41 -0.06

[700 rows x 5 columns]
0     -6.8226793
1     -6.3262905
2     -9.3027282
3     -7.3718925
4     -6.0276470
         ...    
695   -6.1473937
696   -6.2148989
697   -6.8413411
698   -6.5233707
699   -5.6766827
Name: y, Length: 700, dtype: float64
(700, 5)
(700,)


Wir verlieren etwas präzision durch das Laden in den DataFrame (durch float precision)

In [3]:
def transform(X):
    l1 = X
    l2 = X*X
    l3 = np.exp(X)
    l4 = np.cos(X)
    const = np.ones((X.shape[0],1))
    return np.concatenate((l1, l2, l3, l4, const), axis=1)

In [4]:
transformer = FunctionTransformer(transform, validate=True)

In [5]:
#X_Test = np.array([[1, 2, 3, 4, 5], [1, 1,1, 1,1], [1, 1,1, 1,1], [1, 1,1, 1,1]])
#print(transformer.transform(X_Test))

In [6]:
X_trans = transformer.transform(X)
print(X_trans.shape)
#print(X_trans)

(700, 21)


In [7]:
param_grid = {'alpha' : np.logspace(-2, 10, num=200)}
kf = KFold(n_splits=10)
model = Ridge(alpha=1, fit_intercept=False)

grid_search = GridSearchCV(model, param_grid, cv=kf, scoring='neg_root_mean_squared_error', return_train_score=True)

In [8]:
best_model = grid_search.fit(X_trans, Y)
print("Lowest RMSE:" + str(best_model.best_score_))
best_alpha = best_model.best_estimator_
print("With alpha: " + str(best_alpha))

coefficiants = best_model.best_estimator_.coef_
print("And coefficiants. ")
print(coefficiants)

Lowest RMSE:-1.9517431060906634
With alpha: Ridge(alpha=41.504047578504725, fit_intercept=False)
And coefficiants. 
[ 0.0634288  -0.09489264 -0.14181013  0.19592706  0.05507503 -0.05101589
  0.016873    0.02516042 -0.0809805   0.0031407  -0.55958855 -0.68431942
 -0.72761617 -0.43817214 -0.54127371 -0.5717427  -0.60547412 -0.60959287
 -0.55722284 -0.59850169 -0.5970953 ]


store


In [9]:
dictionary = {'Coefficiants': coefficiants}
avg_acc_df = pd.DataFrame(dictionary )
print(avg_acc_df)
avg_acc_df.to_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\submission1b.csv', index=False, header=False)

    Coefficiants
0      0.0634288
1     -0.0948926
2     -0.1418101
3      0.1959271
4      0.0550750
5     -0.0510159
6      0.0168730
7      0.0251604
8     -0.0809805
9      0.0031407
10    -0.5595885
11    -0.6843194
12    -0.7276162
13    -0.4381721
14    -0.5412737
15    -0.5717427
16    -0.6054741
17    -0.6095929
18    -0.5572228
19    -0.5985017
20    -0.5970953


In [14]:
model2 = Ridge(alpha=5.170920242896756)
best_fit = model2.fit(X_trans, Y)
coef = best_fit.coef_
print(coef)

[-0.06612788 -0.68231224 -0.83384997 -0.05459609 -0.04157483 -0.24778276
  0.19426473  0.16548507 -0.03504243  0.10998607 -0.19867299 -0.5892195
 -0.75919001 -0.08069978  0.00312958  0.12313297 -0.0966012  -0.08219195
  0.01657758 -0.05327217  0.        ]


Overfitting Data This does not work

In [17]:
model3 = LinearRegression()
best_fit = model3.fit(X_trans, Y)
coef = best_fit.coef_
print(coef)

[   87.19891672   -67.27010868    28.53822965   158.07550392
  -819.69642957   200.04958807  -393.54175642  1953.26216011
  -523.63288817  1518.93422167   -86.60565699    65.34027498
   -30.9921308   -155.77167729   822.10809668   327.109988
  -737.64020339  3890.67031218 -1211.22831197  3799.82175151
     0.        ]


Versuche nochmals 10 fold cross validation

In [10]:
k = 10
kf = KFold(n_splits=k, random_state=None)
model4 = Ridge(alpha=5.170920242896756, fit_intercept=False)
acc_score = []
best_acc = 1000


In [11]:
for train_index , test_index in kf.split(X):
        X_train , X_test = X_trans[train_index,:],X_trans[test_index,:]
        Y_train , Y_test = Y[train_index] , Y[test_index]

        model4.fit(X_train,Y_train)
        pred_values = model4.predict(X_test)

        acc = np.sqrt(mean_squared_error(pred_values , Y_test))
        if acc < best_acc:
            best_acc = acc
            best_model = model4
            
        acc_score.append(acc)

In [12]:
print(best_acc)
print(acc_score)

1.4307405772067234
[1.843573765736639, 1.8699164762866314, 2.057294075425951, 2.026010242991757, 1.4307405772067234, 2.204681217568447, 2.091630160510509, 1.9260351914411367, 2.087788569124632, 1.9942842920279915]


In [13]:
print(model4.coef_)

[ 0.16527062 -0.48866734 -0.66385665  0.16679502  0.13286039 -0.29960782
  0.09717803  0.13021336 -0.14143101  0.06129912 -0.51987876 -0.97407186
 -1.13894244 -0.43920514 -0.37836275 -0.38525795 -0.58206931 -0.59837934
 -0.46457019 -0.562893   -0.53373116]


In [14]:
dictionary = {'Coefficiants': model4.coef_}
avg_acc_df = pd.DataFrame(dictionary )
print(avg_acc_df)
avg_acc_df.to_csv(r'C:\Users\erics\Documents\Programme\IntroML\Task1\submission1bV4.csv', index=False, header=False)

    Coefficiants
0      0.1652706
1     -0.4886673
2     -0.6638566
3      0.1667950
4      0.1328604
5     -0.2996078
6      0.0971780
7      0.1302134
8     -0.1414310
9      0.0612991
10    -0.5198788
11    -0.9740719
12    -1.1389424
13    -0.4392051
14    -0.3783627
15    -0.3852579
16    -0.5820693
17    -0.5983793
18    -0.4645702
19    -0.5628930
20    -0.5337312


First using Pandas the data is loaded into a dataframe and then split into X (inputs) and Y (the expected output). Using scikit we can define the ridge regression model and the function KFold that splits our dataset into partitions of the same size. We don't use randomness or shuffeling on the dataset since we have enough datapoints to learn from and that it stays reproducible. 
Using the FunctionTransformer the input is transformed as mentioned in the task description. 
We then perform a Gridsearch to find the best alpha parameter for ridge regression. After having found the best alpha we use this alpha to compute the model wheights using ridge regression and a 10-fold cross validation approach. Gridseach also gives us model parameters but since Gridsearch uses rounded values during computation, to speed up computation, we get better wheights doing the learning again as axact as possible. We could take some data out for testing purposes but this is not done here. 
The model parameters can be read out using .coef_ and then saved to csv via a conversion to pandas.dataframe.